In [ ]:
import numpy as np
import pandas as pd
import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt

import seaborn as sns
from parser import *
from visualizer import *
from ya_loader import *

### базовая предложеннная модель

$$f(x_1, x_2, x_3, x_4) = \hat{y} $$

In [ ]:
class baseline_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(4, 8)
        self.l2 = nn.Linear(8, 6)
        self.l3 = nn.Linear(6, 4)
        self.l4 = nn.Linear(4, 3)
        self.l5 = nn.Linear(3, 1)

    def forward(self, x):
        out1 = self.l1(x)
        out2 = self.l2(out1)
        out3 = F.tanh(self.l3(out2))
        out4 = self.l4(out3)
        out5 = self.l4(out4)
        return out5


#### Улучшения модели

Почему бы не кинуть кэтбуст вместо второго и остальных линейных слоёв, мб он даст лучше метрики, на 4 фичах всё будет плохо, но на 8 мб всё ок должно быть 

Можно сделать датафрейм из с помощью парсера, но для каждой фигуры свой датафрейм, как я понял, и тогда для каждой фигуры своя архитектура, то есть наша работа будет бесполезной

#### train

In [ ]:
def train(model, optimizer, scheduler, loss_fn, train_loader, val_loader, n_epochs, device):
    train_losses = []
    val_losses = []

    for epoch in tqdm(range(n_epochs)):
        model.train()
        train_losses_per_epoch = []
        for X_batch, y_bathch in (train_loader):
            optimizer.zero_grad()
            preds = model(X_batch.to(device))
            loss = loss_fn(preds, y_bathch.to(device))
            loss.backward()
            optimizer.step()
            train_losses_per_epoch.append(loss.item())
        scheduler.step()  ### ничего про него я не заметил, пока пусть будет
        train_losses.append(np.mean(train_losses_per_epoch))

        model.eval()
        val_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_bathch in val_loader:
                pred = model(X_batch.to(device))
                loss = loss_fn(pred, y_bathch.to(device))
                val_losses_per_epoch.append(loss.item())

        val_losses.append(np.mean(val_losses_per_epoch))
    
    return train_loader, val_loader

In [ ]:
def predict(model, test_loader, device):
    preds = np.array([])
    with torch.no_grad():
        for X_batch in test_loader:
            pred = model(X_batch.to(device))
            np.append(preds, pred.detach().numpy())
    return preds

#### dataset

In [ ]:
class coords_param(Dataset):
    def __init__(self, made_dataset):
        self.__lenght = 0
        self.data = made_dataset
        super().__init__()

    def __len__(self):
        return self.__lenght
    
    def __getitem__(self, index):
        return self.data.iloc(index)
'''я пока хз какой будет датасет на входе c скорее всего всё совсем не так'''

вообще из того, что есть надо сначала собрать датасет и что-то подумать че и как, щас я вообще хз

Короче нам надо как-то датасет собрать в пандасе, щас это всё протс по преколу, я не ебу зачем они нас заставляют ебаться с этим

### примеры пасера и визуалайзера и лоадера

#### parcer

In [ ]:
PATH_TO_CASE = 'data/princess_luna/0.3M/'
END_TIME = '150'

In [ ]:
base_path = Path(PATH_TO_CASE)
time_path = base_path / Path(END_TIME)
p_path = time_path / Path('p')
p = Ofpp.parse_internal_field(p_path)

In [ ]:
surface = pressure_field_on_surface(base_path, p)

In [ ]:
for s in surface[0]['Item2']:
    pprint(s)
    break

#### visualizer

In [ ]:
plot_slices(base_path, base_path)

#### loader

In [ ]:
yadisk_url = 'https://disk.yandex.ru/i/3ywDfjugWiULSQ'
print(get_real_direct_link(yadisk_url))

In [ ]:
urllib.request.urlretrieve(get_real_direct_link(yadisk_url), 'data.jpg') 

                                                        link, path_for_img